# Spidroin curation
设置需要用到的 Python 包

In [2]:
import os

设置工作目录：

In [5]:
# 查看当前路径
os.getcwd()

# 设置工作目录
os.chdir("/home/gyk/project/spider_silkome")
os.getcwd()

'/home/gyk/project/spider_silkome'

定义相关数据的路径

In [6]:
spider_species_file = "data/external/organisms.csv"
mechanical_properties_file = "data/external/mechanical_properties.csv"
spidroin_fasta_file = "data/external/spider-silkome-database.v1.prot.fasta"

根据物种名拆分丝蛋白序列